# Dataclasses

Python dataclasses (Python 3.7+) automatically generate common methods for data-focused classes, dramatically reducing boilerplate code.

```python
# Traditional approach → verbose
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __repr__(self):
        return f"Person(name={self.name!r}, age={self.age!r})"

    def __eq__(self, other):
        return isinstance(other, Person) and self.name == other.name and self.age == other.age

# Dataclass approach → clean and simple
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    age: int
```

## Auto-Generated Methods

The `@dataclass` decorator automatically creates:

* `__init__()` – Constructor
* `__repr__()` – String representation
* `__eq__()` – Equality comparison
* (optionally) `__lt__`, `__le__`, `__gt__`, `__ge__` if `order=True`


## Field Features

```python
from dataclasses import dataclass, field

@dataclass
class Product:
    name: str                                    # Required field
    price: float = 0.0                           # Default value
    tags: list = field(default_factory=list)     # Safe mutable default
    internal_id: str = field(init=False)         # Excluded from __init__
```

* `default_factory` is used for mutable defaults (e.g., list, dict).
* `init=False` means the field won't be included in the generated `__init__`.

## The `__post_init__` Method

**What it is:** A special method that runs automatically after `__init__`.

**When it runs:** After the dataclass constructor finishes initializing fields.

```python
@dataclass
class Circle:
    radius: float
    area: float = field(init=False)

    def __post_init__(self):
        self.area = 3.14159 * self.radius ** 2
```

## `__post_init__` vs `@property`

| Feature     | `__post_init__`                  | `@property`                          |
| ----------- | -------------------------------- | ------------------------------------ |
| Computation | Once at object creation          | On every access                      |
| Performance | Faster after initial computation | Slightly slower (computed each time) |
| Storage     | Stores result in attribute       | Does not store, always recalculated  |
| Staleness   | Can become outdated              | Always up-to-date                    |

## `InitVar` (Initialization Variables)

**Purpose:** Used to accept values during initialization that are **not stored** as instance attributes.

```python
from dataclasses import dataclass, field, InitVar

@dataclass
class Person:
    name: str
    birth_year: int
    current_year: InitVar[int]  # Passed to __post_init__, not stored
    age: int = field(init=False)

    def __post_init__(self, current_year):
        self.age = current_year - self.birth_year

person = Person("Alice", 1990, 2024)
# print(person.current_year)  # ❌ AttributeError: 'Person' object has no attribute 'current_year'
print(person)  # Person(name='Alice', birth_year=1990, age=34)
```

## Advanced Features

```python
@dataclass(frozen=True, order=True, slots=True)
class Point:
    x: float
    y: float
```

* `frozen=True`: Makes the instance immutable
* `order=True`: Adds ordering methods (`__lt__`, etc.)
* `slots=True`: Optimizes memory and speeds up attribute access (Python 3.10+)

### More Field Customization

```python
@dataclass
class Student:
    grades: list = field(default_factory=list)
    compare_key: str = field(compare=False)  # Excluded from equality and ordering
```

## Useful Utilities

```python
from dataclasses import asdict, astuple

@dataclass
class Book:
    title: str
    author: str

b = Book("1984", "George Orwell")
print(asdict(b))   # {'title': '1984', 'author': 'George Orwell'}
print(astuple(b))  # ('1984', 'George Orwell')
```

## When to Use Dataclasses

Use `@dataclass` when:

* You need a class primarily for storing data
* You want automatic `__init__`, `__repr__`, etc.
* You want cleaner, more readable code
* You're handling structured data (e.g. from APIs, configs)
* You don't need complex inheritance or custom metaclasses

In [1]:
# Example-1
# Too much boilerplate
class Person:
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email
    
    def __repr__(self):
        return f"Person(name='{self.name}', age={self.age}, email='{self.email}')"
    
    def __eq__(self, other):
        if not isinstance(other, Person):
            return False
        return (self.name, self.age, self.email) == (other.name, other.age, other.email)
    
# Reduced boilerplate
from dataclasses import dataclass, field, InitVar

@dataclass
class Person:
    name: str
    age: int
    email: str

In [2]:
person1 = Person(name='Prakhar', age=20, email='prhkrd@icloud.com')
person2 = Person(name='Prakhar', age=20, email='prhkrd@icloud.com')
person1 == person2

True

In [3]:
# Example-2: Post-initialization
@dataclass
class Product:
    name: str
    price: float
    category: str = "General"
    in_stock: bool = True
    tags: list = None

    def __post_init__(self):
        if self.tags is None:
            self.tags = []

product = Product(name='A', price=2.3)
print(product.tags)

[]


In [4]:
@dataclass(frozen=True)
class Point:
    x: float
    y: float

point = Point(x=1, y=2)
point.x = 2

FrozenInstanceError: cannot assign to field 'x'

In [5]:
# Example-3: Post initialization
from dataclasses import dataclass, field

@dataclass
class Student:
    name: str
    grades: list = field(default_factory=list)  # Safe mutable default
    id: str = field(init=False)  # Not included in __init__
    
    def __post_init__(self):
        self.id = f"STU_{hash(self.name) % 10000}"

student = Student(name='Jan', grades=['A', 'C', 'D'])
print(student)

Student(name='Jan', grades=['A', 'C', 'D'], id='STU_9064')


In [6]:
# Example-4: initialization variables
@dataclass
class Rectangle:
    width: float
    height: float
    multiplier: InitVar[float]
    area: float = field(init=False)
    
    def __post_init__(self, multiplier):
        self.area = self.width * self.height * multiplier

rec = Rectangle(width=1, height=1, multiplier=2)
print(rec)

Rectangle(width=1, height=1, area=2)
